In [1]:
from PIL import Image
import numpy as np
import glob
import cv2 as cv
import os
import numpy as np
import sklearn as sk #general imports, initial data preprocessing/OS stuff
import os
import torch as T
import torch.nn as nn
import torch.optim as optim #Neural network imports, multiply data etc
from torchvision.transforms import ToTensor
import torchvision.models as models
import torchvision
import torch.nn.functional as F #Neural Network used in Comp4660 at ANU
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from pathlib import Path
from torchvision.datasets import ImageFolder
from sklearn.preprocessing import MinMaxScaler #normalize data
from sklearn.metrics import confusion_matrix #analysis
from torchvision import transforms
from random import shuffle
import itertools

C:\Users\campb\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\campb\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
### - GLOBALS - ###

DPS = '/Users/campb/Documents/Comp4528/CLab-2/Dataset' #Change for marking my computer just isnt great with paths as im on windows

folds = 10

device = T.device('cuda:0' if (T.cuda.is_available())  else 'cpu') #Train on device, no parallel gpu

In [3]:
train_ims = np.load('/Users/campb/Documents/Comp4528/CLab-2/Dataset/kmnist-train-imgs.npz')['arr_0']
test_ims = np.load('/Users/campb/Documents/Comp4528/CLab-2/Dataset/kmnist-test-imgs.npz')['arr_0']
train_lab = np.load('/Users/campb/Documents/Comp4528/CLab-2/Dataset/kmnist-train-labels.npz')['arr_0']
test_lab = np.load('/Users/campb/Documents/Comp4528/CLab-2/Dataset/kmnist-test-labels.npz')['arr_0']
val_ims = np.load('/Users/campb/Documents/Comp4528/CLab-2/Dataset/kmnist-val-imgs.npz')['arr_0']
val_lab = np.load('/Users/campb/Documents/Comp4528/CLab-2/Dataset/kmnist-val-labels.npz')['arr_0']

In [4]:
print(train_ims.shape, train_lab.shape, test_ims.shape, test_lab.shape, val_ims.shape, val_lab.shape)

(59000, 28, 28) (59000,) (10000, 28, 28) (10000,) (1000, 28, 28) (1000,)


In [5]:
#Import as tensors
train = T.from_numpy(np.float32(np.concatenate([train_ims,test_ims])))
labs = T.from_numpy(np.float32(np.concatenate([train_lab,test_lab])))
assert(train.shape[0] == labs.shape[0])
val_ims = T.from_numpy(np.float32(val_ims))
val_lab = T.from_numpy(np.float32(val_lab))

In [6]:
#Data augmentation
transform = transforms.Compose([
            transforms.Normalize((0.), (1.0)),
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomCrop(28, 4) #Random crop image with output size 28 and padding set to 4, default padding is 0 padding
            ])

In [7]:
train = transform(train) #apply transform to train images

In [8]:
val_ims = transform(val_ims) #apply transform to validation images

In [9]:
print(train.shape, labs.shape, val_ims.shape, val_lab.shape)

### - Model Variables - ###
lr = 1e-3
bs = (0.9, 0.999)

LabNet= nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
    nn.ReLU(), #First layer
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(), #Second layer
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(7*7*64, 1024), #Filter size 64, and we downsample 2x twice so 28/4
    nn.ReLU(),
    nn.Linear(1024,10)
)

print(LabNet)

torch.Size([69000, 28, 28]) torch.Size([69000]) torch.Size([1000, 28, 28]) torch.Size([1000])
Sequential(
  (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=3136, out_features=1024, bias=True)
  (8): ReLU()
  (9): Linear(in_features=1024, out_features=10, bias=True)
)


In [10]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(LabNet.parameters(), lr = lr, betas=bs)

In [11]:
epochs = 10

#Loss and Accuracy Methods Only need train
all_tloss_e = []
all_tacc_e = []

#Setup k-Fold
d = list(range(train.shape[0]))
shuffle(d) #Data is shuffled
d = np.array_split(d, folds)

In [38]:
### - train cell - ###
for epoch in range(epochs):
    correct_train = 0
    total_train = 0
    for i in range(folds-1):
        test = train[d[-i]]
        t_lab = labs[d[-i]]

        train_i = d[:i] + d[i+1:]
        train_set = train[list(itertools.chain.from_iterable(train_i))]
        train_lab = labs[list(itertools.chain.from_iterable(train_i))]
        
        output_train = []
        
        correct = 0
        
        for i, im in enumerate(train_set):
            outs = LabNet(im[None,:].unsqueeze(1)) #forward pass
            loss_i = loss(outs, train_lab[i].unsqueeze(-1).long())
            
            all_tloss_e.append(loss_i) #append losses to graph
            
            output_train.append(outs)
            
            LabNet.zero_grad(set_to_none=True) # Reset Grads
            
            loss_i.backward()
            
            optimizer.step()
            
            #check if correct
            guess = T.argmax(outs).item()
            if guess == train_lab[i].item():
                correct += 1
    

8
8
8
8
8
8
8
6
6
9
9
3
3
5
5
2
2
4
4
4
2
4
2
1
4
2
1
4
1
2
4
1
1
1
1
1
1
1
1
1
0
5
1
1
2
4
1
1
4
4
5
6
1
6
4
9
4
4
4
4
4
2


KeyboardInterrupt: 